Train a model to predict complete loss of methylation or partial loss using a sequence

In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [3]:
cd /gdrive/My\ Drive/nn 

/gdrive/My Drive/nn


In [4]:
%tensorflow_version 1.x
import argparse
import os
import pickle
import sys
import glob

import numpy as np

np.random.seed(7)  # for reproducibility

import tensorflow as tf
tf.random.set_random_seed(5005)

from sklearn.model_selection import train_test_split, KFold
from sklearn.utils import class_weight


from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.layers import Dense, Flatten, Dropout
from tensorflow.python.keras.layers.convolutional import Conv1D
from tensorflow.python.keras.layers.pooling import MaxPooling1D
from tensorflow.python.keras.layers.pooling import AveragePooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow.python.keras.backend as K
from keras import regularizers
from tensorflow.python.keras.utils import plot_model 

sys.path.append(".")
import utils
from utils import precision, recall_TP, load_train_validate_test_data, recall_TN, precision_N

l2_lam = 5e-07 
l1_lam = 1e-08 

TensorFlow 1.x selected.


Using TensorFlow backend.


In [5]:
def train_model_on_fold(x_train, y_train, x_test,y_test, input_len,
                        num_epoch, batchsize, func,model_path):
  """
  Train a model to using the train data to predict the test data
  :param x_train: The train dataset 
  :param y_train: The train labels
  :param x_test: The test dataset
  :param y_test: The test labels
  :param input_len: The length of the input
  :param num_epoch: Number of epoches 
  :param batchsize: The batchsize 
  :param func: The model function to use 
  :param model_path: The path to save the model from run to run
  :return: The model after fitting
  """
  model = func(input_len)
  adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6)
  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy', recall_TP,recall_TN])
  checkpointer = ModelCheckpoint(filepath=model_path, verbose=1, save_best_only=True)
  earlystopper = EarlyStopping(monitor='val_loss', patience=5, verbose=2)
    
  print('fitting the model')          
  sample_weights = class_weight.compute_sample_weight('balanced', y_train)

  history = model.fit(x_train, y_train, epochs=num_epoch, batch_size=batchsize,
                      validation_data=(x_test, y_test), verbose=1,
                      callbacks=[checkpointer, earlystopper, ], sample_weight=sample_weights)
  return model

In [6]:
def sequence_model(input_len):
  """
  Buld a model to predict a sequence information 
  :param input_len: The length of the input
  """
  K.clear_session()
  tf.random.set_random_seed(5005)

  input_node = Input(shape=(input_len, 4), name="input")
  conv1 = Conv1D(filters=90, kernel_size=3, padding='valid', activation="relu", name="conv1",kernel_regularizer=regularizers.l2(l2_lam))(input_node)
  pool1 = MaxPooling1D(pool_size=2, strides=1, name="pool1")(conv1)
  drop1 = Dropout(0.25, name="drop1")(pool1)

  conv2 = Conv1D(filters=100, kernel_size=5, padding='valid', activation="relu", name="conv2", kernel_regularizer=regularizers.l2(l2_lam))(drop1)
  pool2 = MaxPooling1D(pool_size=2, strides=1)(conv2)
  drop2 = Dropout(0.25)(pool2)
  flat = Flatten()(drop2)

  hidden1 = Dense(500, activation='relu', name="hidden1",kernel_regularizer=regularizers.l1(l1_lam))(flat)
  drop3 = Dropout(0.5)(hidden1)
  hidden2 = Dense(250, activation='relu', name="hidden2",kernel_regularizer=regularizers.l1(l1_lam))(drop3)

  output = Dense(1, activation='sigmoid', name="output")(hidden2)
  model = Model(inputs=[input_node], outputs=output)

  return model

In [7]:
def train_model(data_path, model_folder="./models/folds_models", temp_model_folder="./models/temp/weight.h5", input_len=150, number_of_folds=5):
  """
  Train a model or x models using the cross validation 
  :param data_path: The path for the dataset
  :param model_folder: The final folder to save the models
  :param temp_model_folder: A folder to save the models while running
  :param input_len: The length of the input
  :param number_of_folds: Number of fold to use for the model
  :return: The model if we used 1 model(1 fold) or None if more 
  """

  print('loading data')
  x_train_list, y_train_list, x_valid_list, y_valid_list, x_test_seq, y_test = load_train_validate_test_data(data_path, input_len, kfold=number_of_folds)

  models_path = []
  acc_per_fold = []
  loss_per_fold = []

  for fold_num in range(len(x_train_list)):
    print("Using fold %s/%s" %(fold_num+1, number_of_folds))
    x_train_seq = x_train_list[fold_num]
    y_train = y_train_list[fold_num]
    x_valid_seq = x_valid_list[fold_num]
    y_valid = y_valid_list[fold_num]

    temp_model_file = model_path  = os.path.join(model_folder, "fold%s.h5" %fold_num)

    model = train_model_on_fold(x_train_seq, y_train, x_valid_seq, y_valid, model_path=temp_model_folder, 
                            input_len=150, num_epoch=20, batchsize=128, func = sequence_model)
    
    if fold_num == 0:
      print(model.summary())
      plot_model(model, show_shapes=True, show_layer_names=True,rankdir="TB")

    print("Finish training fold %d" % (fold_num+1))
    print('testing the model')
    score = model.evaluate(x_test_seq, y_test)

    for i in range(len(model.metrics_names)):
        print(str(model.metrics_names[i]) + ": " + str(score[i]))

    acc_per_fold.append(score[1] * 100)
    loss_per_fold.append(score[0])
    models_path.append(model_path)

    model.save(model_path)

  print('Average scores for all folds:')
  print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
  print(f'> Loss: {np.mean(loss_per_fold)}')

  if number_of_folds == 1:
    return model
  
  return None 

In [ ]:
def load_models(models_folder):
  """
  Load the different models from the folder
  :param models_folder: path to the folder with models 
  """
  models_paths = glob.glob(os.path.join(models_folder, "*.h5"))

  models = [load_model(model_path, custom_objects={'recall_TP': recall_TP,'recall_TN': recall_TN }) for model_path in models_paths]
  return models


def get_scores(models, x_test, y_test):
  """
  Calculate and print the scores (accuracy and loss) for every model and the accuracy value
  :param models: A list of loaded models 
  :param x_test: The dataset 
  :param y_test: The real labels
  :return A list of accuracy scores and loss scores per fold
  """
  acc_per_fold =[]
  loss_per_fold = []

  for model in models:
    score = model.evaluate(x_test_seq, y_test)
    acc_per_fold.append(score[1] * 100)
    loss_per_fold.append(score[0])

  print('Average scores for all folds:')
  print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
  print(f'> Loss: {np.mean(loss_per_fold)}')
  return acc_per_fold, loss_per_fold

def predict(models, x, use_majority=True):
  """
  Predict the output of x using the different models that were provided, if use_majority=True we use majority vote, if False we use Average vote
  :param models: A list of loaded models 
  :param x: The dataset to predict on
  :param use_majority:if use_majority=True we use majority vote, if False we use Average vote
  :return the prediction for labels 
  """
  y_pred = np.zeros(shape=(x.shape[0], len(models)))
  
  for i in range(len(models)):
    if use_majority:
      model_prediciton = np.round(models[i].predict(x))    
    else:
      model_prediciton = models[i].predict(x)

    y_pred[:,i] = model_prediciton.reshape(model_prediciton.shape[0])
    
  return np.round(np.mean(y_pred, axis=1))

Run the different functions

In [ ]:
# Train the NN on the scWGBS data using 5 folds 
zhou_all_data = r"dataset/zhou_all.pkl"
zhou_solo_data = r"dataset/zhou_solo.pkl"
scgwbs_data = r"dataset/scwgbs_solo.pkl"
models_folder="./models/folds_models"

In [ ]:
# Train
model = train_model(data_path=data_path, model_folder=models_folder, temp_model_folder="./models/temp/weight.h5", input_len=150, number_of_folds=2)

In [ ]:
# Test
_,_,_,_, x_test_seq, y_test = load_train_validate_test_data(path_to_data=scgwbs_data, input_len=150, kfold=1, only_test=True)

models = load_models(models_folder)
_, _ = get_scores(models,x_test_seq, y_test)

# Get global prediction
y_pred = predict(models, x_test_seq)

# Get accuracy of combined model
diff = y_test - y_pred
good_predictions = np.sum(diff == 0)
print("Accuracy using majority vote: %s" %(good_predictions / y_test.shape[0] * 100))